<a href="https://colab.research.google.com/github/abdobeyaye/E-bookauthor/blob/main/E_bookauthor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-

import os
from getpass import getpass
from groq import Groq
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.section import WD_SECTION
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import time
import random
import re

def safe_input(prompt):
    return input(prompt).encode('utf-8').decode('utf-8')

# Securely input your API key
api_key = getpass("Enter your Groq API key: ")

# Initialize the Groq client
client = Groq(api_key=api_key)

def generate_content(prompt, max_attempts=3):
    prompt = prompt.encode('utf-8').decode('utf-8')
    for attempt in range(max_attempts):
        try:
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a highly skilled author and book organizer. Your task is to create well-structured, engaging content with clear hierarchies and organization."
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model="llama-3.1-70b-versatile",
                temperature=0.7,
                max_tokens=4000,
                top_p=1,
                stream=False,
                stop=None
            )
            if chat_completion.choices[0].message.content:
                return chat_completion.choices[0].message.content
            else:
                print(f"Warning: Empty response on attempt {attempt + 1}.")
        except Exception as e:
            print(f"Error on attempt {attempt + 1}: {str(e)}")
            if attempt == max_attempts - 1:
                raise
        time.sleep(random.uniform(1, 3))
    return "Failed to generate content after multiple attempts."

def generate_book_title(topic, style, author):
    prompt = f"""Generate a single, poetic and engaging title for a book about {topic} written in the style of {style} by {author}.
    The title should be evocative, relevant to the topic and style, and capture the essence of the story.
    Provide only the title, without any additional explanation."""
    return generate_content(prompt)

def generate_book_outline(topic, style, num_chapters, author):
    prompt = f"""Create a detailed outline for a {num_chapters}-chapter book about {topic} in the style of {style}, written by {author}.
    The outline should include:
    1. A brief introduction to the book's theme
    2. For each chapter:
       a. A poetic chapter title
       b. A brief synopsis (2-3 sentences)
       c. 3-5 subheadings or key sections within the chapter
       d. Key plot points or concepts to be covered
       e. Character development or thematic elements to focus on
    3. An epilogue or conclusion section
    Use rich, poetic language throughout the outline. Ensure a clear and logical progression of ideas across chapters."""
    return generate_content(prompt)

def generate_chapter(topic, style, chapter_number, chapter_outline, author):
    prompt = f"""Write chapter {chapter_number} of a book about {topic} in the style of {style}, authored by {author}.
    Use the following outline as a guide:

    {chapter_outline}

    The chapter should be approximately 1000-1200 words long (about 4 pages), rich in descriptive language, vivid imagery, and emotional depth.
    Focus on creating a powerful atmosphere that captures the essence of the story.
    Include dialogue, descriptive passages, and character thoughts as appropriate for the style and topic.

    Structure the chapter with clear sections:
    1. Start with the chapter title
    2. Use 3-5 subheadings throughout the chapter to divide major sections
    3. Under each subheading, write 1-2 paragraphs of content
    4. Ensure smooth transitions between sections
    5. End the chapter with a compelling hook or cliffhanger

    Use varied sentence structures and paragraph lengths to maintain reader engagement.
    Remember to keep the total length to about 4 pages (1000-1200 words)."""
    return generate_content(prompt)

def generate_front_matter(book_title, author, year):
    prompt = f"""Create front matter for a book titled "{book_title}" by {author}, published in {year}. Include:
    1. A short, engaging dedication (1-2 sentences)
    2. A brief, poetic epigraph (a quote that captures the essence of the book)
    3. A preface (about 200 words) that introduces the book's theme and the author's motivation
    4. An acknowledgments section (about 150 words) thanking key individuals or sources of inspiration

    Use language that matches the book's style and theme.
    Start each section with its title (e.g., "Dedication:", "Epigraph:", etc.) on a new line."""
    front_matter = generate_content(prompt)

    # Initialize default values
    parsed_front_matter = {
        "dedication": "",
        "epigraph": "",
        "preface": "",
        "acknowledgments": ""
    }

    # Parse the generated content into sections
    current_section = None
    for line in front_matter.split('\n'):
        if line.lower().startswith("dedication:"):
            current_section = "dedication"
        elif line.lower().startswith("epigraph:"):
            current_section = "epigraph"
        elif line.lower().startswith("preface:"):
            current_section = "preface"
        elif line.lower().startswith("acknowledgments:"):
            current_section = "acknowledgments"
        elif current_section:
            parsed_front_matter[current_section] += line.strip() + "\n"

    # Trim any trailing newlines
    for key in parsed_front_matter:
        parsed_front_matter[key] = parsed_front_matter[key].strip()

    return parsed_front_matter

def generate_back_matter(book_title, author, topic):
    prompt = f"""Create back matter for the book "{book_title}" by {author} about {topic}. Include:
    1. An afterword (about 300 words) reflecting on the book's journey and its significance
    2. A glossary of 10-15 key terms or concepts related to the book's theme
    3. A brief author biography (about 150 words)
    4. Suggestions for further reading (5-7 books or articles related to the topic)

    Ensure the content is engaging and relevant to the book's theme.
    Start each section with its title (e.g., "Afterword:", "Glossary:", etc.) on a new line."""
    back_matter = generate_content(prompt)

    # Initialize default values
    parsed_back_matter = {
        "afterword": "",
        "glossary": "",
        "author_bio": "",
        "further_reading": ""
    }

    # Parse the generated content into sections
    current_section = None
    for line in back_matter.split('\n'):
        if line.lower().startswith("afterword:"):
            current_section = "afterword"
        elif line.lower().startswith("glossary:"):
            current_section = "glossary"
        elif line.lower().startswith("about the author:"):
            current_section = "author_bio"
        elif line.lower().startswith("further reading:"):
            current_section = "further_reading"
        elif current_section:
            parsed_back_matter[current_section] += line.strip() + "\n"

    # Trim any trailing newlines
    for key in parsed_back_matter:
        parsed_back_matter[key] = parsed_back_matter[key].strip()

    return parsed_back_matter

def sanitize_filename(filename):
    sanitized = re.sub(r'[<>:"/\|?*]', '', filename)
    return sanitized[:200]  # Limit to 200 characters

def write_book(author, year):
    topic = safe_input("Enter the book topic or description: ")
    style = safe_input("Enter the writing style (e.g., 'novel', 'lyrical'): ")
    num_chapters = int(safe_input("Enter the number of chapters for the book: "))

    book_title = generate_book_title(topic, style, author)
    print(f"Generated book title: {book_title}")

    print("Generating book outline...")
    book_outline = generate_book_outline(topic, style, num_chapters, author)
    print("Book outline generated.")

    print("Generating front matter...")
    front_matter = generate_front_matter(book_title, author, year)
    print("Front matter generated.")

    chapters = []
    for chapter in range(1, num_chapters + 1):
        print(f"Generating chapter {chapter}...")
        chapter_outline = extract_chapter_outline(book_outline, chapter)
        chapter_content = generate_chapter(topic, style, chapter, chapter_outline, author)
        chapters.append(chapter_content)
        time.sleep(random.uniform(2, 5))

    print("Generating back matter...")
    back_matter = generate_back_matter(book_title, author, topic)
    print("Back matter generated.")

    return book_title, book_outline, front_matter, chapters, back_matter

def extract_chapter_outline(book_outline, chapter_number):
    chapter_start = book_outline.find(f"Chapter {chapter_number}")
    if chapter_start == -1:
        return "Chapter outline not found."
    next_chapter = book_outline.find(f"Chapter {chapter_number + 1}", chapter_start)
    if next_chapter == -1:
        return book_outline[chapter_start:]
    return book_outline[chapter_start:next_chapter]

def organize_book_content(book_title, book_outline, front_matter, chapters, back_matter, author, year):
    organized_content = {
        "title": book_title,
        "author": author,
        "year": year,
        "front_matter": front_matter,
        "outline": book_outline,
        "chapters": [],
        "back_matter": back_matter
    }

    for i, chapter in enumerate(chapters, 1):
        chapter_content = {
            "number": i,
            "title": f"Chapter {i}",
            "sections": []
        }

        sections = re.split(r'\n(?=\d+\.)', chapter)
        for section in sections:
            if section.strip():
                lines = section.split('\n')
                section_title = lines[0].strip()
                section_content = '\n'.join(lines[1:]).strip()
                chapter_content["sections"].append({
                    "title": section_title,
                    "content": section_content
                })

        organized_content["chapters"].append(chapter_content)

    return organized_content

def add_page_number(paragraph):
    run = paragraph.add_run()
    fldChar = OxmlElement('w:fldChar')
    fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')
    instrText.text = "PAGE"
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'end')
    run._element.append(fldChar)
    run._element.append(instrText)
    run._element.append(fldChar2)

def add_copyright_page(doc, author, year):
    doc.add_paragraph(f"Copyright © {year} by {author}")
    doc.add_paragraph("All rights reserved.")
    doc.add_paragraph("No part of this book may be reproduced in any form or by any electronic or mechanical means, including information storage and retrieval systems, without written permission from the author, except for the use of brief quotations in a book review.")
    doc.add_page_break()

def add_headers_and_footers(doc, title, author):
    for section in doc.sections:
        header = section.header
        footer = section.footer

        # Add title to header
        header_para = header.paragraphs[0]
        header_para.text = title
        header_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT

        # Add author to footer
        footer_para = footer.paragraphs[0]
        footer_para.text = author
        footer_para.alignment = WD_ALIGN_PARAGRAPH.LEFT

        # Add page number to footer
        page_num_para = footer.add_paragraph()
        page_num_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        add_page_number(page_num_para)

def save_book_as_docx(organized_content, filename=None):
    if filename is None:
        filename = sanitize_filename(organized_content["title"]) + ".docx"

    doc = Document()

    # Set up A4 page size
    section = doc.sections[0]
    section.page_height = Pt(842)  # A4 height in points
    section.page_width = Pt(595)   # A4 width in points
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)
    section.top_margin = Inches(1)
    section.bottom_margin = Inches(1)

    # Set up styles
    styles = doc.styles

    # Title style
    title_style = styles.add_style('CustomTitle', WD_STYLE_TYPE.PARAGRAPH)
    title_style.font.name = 'Garamond'
    title_style.font.size = Pt(36)
    title_style.font.bold = True
    title_style.font.color.rgb = RGBColor(0, 0, 128)  # Navy Blue
    title_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    title_style.paragraph_format.space_after = Pt(12)

    # Chapter style
    chapter_style = styles.add_style('CustomChapter', WD_STYLE_TYPE.PARAGRAPH)
    chapter_style.font.name = 'Georgia'
    chapter_style.font.size = Pt(24)
    chapter_style.font.bold = True
    chapter_style.font.color.rgb = RGBColor(128, 0, 0)  # Maroon
    chapter_style.paragraph_format.space_before = Pt(24)
    chapter_style.paragraph_format.space_after = Pt(12)

    # Section style
    section_style = styles.add_style('CustomSection', WD_STYLE_TYPE.PARAGRAPH)
    section_style.font.name = 'Calibri'
    section_style.font.size = Pt(18)
    section_style.font.bold = True
    section_style.font.color.rgb = RGBColor(0, 100, 0)  # Dark Green
    section_style.paragraph_format.space_before = Pt(18)
    section_style.paragraph_format.space_after = Pt(6)

    # Body text style
    body_style = styles.add_style('CustomBody', WD_STYLE_TYPE.PARAGRAPH)
    body_style.font.name = 'Bookman Old Style'
    body_style.font.size = Pt(11)
    body_style.paragraph_format.first_line_indent = Inches(0.25)
    body_style.paragraph_format.space_after = Pt(6)

    # Heading 1 style
    heading1_style = styles.add_style('CustomHeading1', WD_STYLE_TYPE.PARAGRAPH)
    heading1_style.font.name = 'Arial'
    heading1_style.font.size = Pt(20)
    heading1_style.font.bold = True
    heading1_style.font.color.rgb = RGBColor(70, 130, 180)  # Steel Blue
    heading1_style.paragraph_format.space_before = Pt(24)
    heading1_style.paragraph_format.space_after = Pt(12)

    # Title page
    doc.add_paragraph(organized_content["title"], style='CustomTitle')
    doc.add_paragraph(f"By {organized_content['author']}", style='CustomBody').alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph(str(organized_content["year"]), style='CustomBody').alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    # Add copyright page
    add_copyright_page(doc, organized_content['author'], organized_content['year'])

    # Front Matter
    doc.add_paragraph("Dedication", style='CustomHeading1')
    doc.add_paragraph(organized_content["front_matter"]["dedication"], style='CustomBody')
    doc.add_paragraph("Epigraph", style='CustomHeading1')
    doc.add_paragraph(organized_content["front_matter"]["epigraph"], style='CustomBody')
    doc.add_paragraph("Preface", style='CustomHeading1')
    doc.add_paragraph(organized_content["front_matter"]["preface"], style='CustomBody')
    doc.add_paragraph("Acknowledgments", style='CustomHeading1')
    doc.add_paragraph(organized_content["front_matter"]["acknowledgments"], style='CustomBody')
    doc.add_page_break()

    # Table of Contents
    doc.add_paragraph("Table of Contents", style='CustomHeading1')
    for chapter in organized_content["chapters"]:
        doc.add_paragraph(f"Chapter {chapter['number']}: {chapter['title']}", style='CustomBody')
    doc.add_page_break()

    # Book outline
    doc.add_paragraph("Book Outline", style='CustomHeading1')
    doc.add_paragraph(organized_content["outline"], style='CustomBody')
    doc.add_page_break()

    # Chapters
    for chapter in organized_content["chapters"]:
        doc.add_paragraph(f"Chapter {chapter['number']}: {chapter['title']}", style='CustomChapter')
        for section in chapter["sections"]:
            doc.add_paragraph(section["title"], style='CustomSection')
            doc.add_paragraph(section["content"], style='CustomBody')
        doc.add_page_break()

    # Back Matter
    doc.add_paragraph("Afterword", style='CustomHeading1')
    doc.add_paragraph(organized_content["back_matter"]["afterword"], style='CustomBody')
    doc.add_paragraph("Glossary", style='CustomHeading1')
    doc.add_paragraph(organized_content["back_matter"]["glossary"], style='CustomBody')
    doc.add_paragraph("About the Author", style='CustomHeading1')
    doc.add_paragraph(organized_content["back_matter"]["author_bio"], style='CustomBody')
    doc.add_paragraph("Further Reading", style='CustomHeading1')
    doc.add_paragraph(organized_content["back_matter"]["further_reading"], style='CustomBody')

    # Add headers and footers
    add_headers_and_footers(doc, organized_content['title'], organized_content['author'])

    doc.save(filename)
    print(f"Book saved as {filename}")

if __name__ == "__main__":
    print("Welcome to the Enhanced AI Book Writing Tool using Groq API and llama-3.1-70b-versatile model!")
    try:
        author = safe_input("Enter the author's name: ")
        year = safe_input("Enter the year of writing: ")
        book_title, book_outline, front_matter, chapters, back_matter = write_book(author, year)
        organized_content = organize_book_content(book_title, book_outline, front_matter, chapters, back_matter, author, year)
        save_book_as_docx(organized_content)
        print("Book generation complete!")
    except Exception as e:
        print(f"An error occurred during book generation: {str(e)}")
        print("Please check your inputs and try again.")

Enter your Groq API key: ··········
Welcome to the Enhanced AI Book Writing Tool using Groq API and llama-3.1-70b-versatile model!
Enter the author's name: الشيخ عبدالقادر عقبة
Enter the year of writing: 2024
Enter the book topic or description: قصة الإمبراطورية العثمانية
Enter the writing style (e.g., 'novel', 'lyrical'): Historical 
Enter the number of chapters for the book: 4
Generated book title: "ذكريات دولة الزمن: قصة الإمبراطورية العثمانية من البداية إلى النهاية"
Generating book outline...
Book outline generated.
Generating front matter...
Front matter generated.
Generating chapter 1...
Generating chapter 2...
Generating chapter 3...
Generating chapter 4...
Generating back matter...
Back matter generated.
Book saved as ذكريات دولة الزمن قصة الإمبراطورية العثمانية من البداية إلى النهاية.docx
Book generation complete!


In [ ]:
# -*- coding: utf-8 -*-

import os
from getpass import getpass
from groq import Groq
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.section import WD_SECTION
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import time
import random
import re

def safe_input(prompt):
  return input(prompt).encode('utf-8').decode('utf-8')

# إدخال مفتاح API الخاص بك بأمان
api_key = getpass("أدخل مفتاح Groq API الخاص بك: ")

# تهيئة عميل Groq
client = Groq(api_key=api_key)

def generate_content(prompt, max_attempts=3):
  prompt = prompt.encode('utf-8').decode('utf-8')
  for attempt in range(max_attempts):
      try:
          chat_completion = client.chat.completions.create(
              messages=[
                  {
                      "role": "system",
                      "content": "أنت كاتب ماهر ومنظم كتب. مهمتك هي إنشاء محتوى منظم وجذاب مع هياكل واضحة وتنظيم."
                  },
                  {
                      "role": "user",
                      "content": prompt,
                  }
              ],
              model="llama-3.1-70b-versatile",
              temperature=0.7,
              max_tokens=4000,
              top_p=1,
              stream=False,
              stop=None
          )
          if chat_completion.choices[0].message.content:
              return chat_completion.choices[0].message.content
          else:
              print(f"تحذير: استجابة فارغة في المحاولة {attempt + 1}.")
      except Exception as e:
          print(f"خطأ في المحاولة {attempt + 1}: {str(e)}")
          if attempt == max_attempts - 1:
              raise
      time.sleep(random.uniform(1, 3))
  return "فشل في إنشاء المحتوى بعد عدة محاولات."

def generate_book_title(topic, style, author):
  prompt = f"""أنشئ عنوانًا شعريًا وجذابًا لكتاب عن {topic} مكتوب بأسلوب {style} بواسطة {author}.
  يجب أن يكون العنوان معبرًا ومرتبطًا بالموضوع والأسلوب، ويعكس جوهر القصة.
  قدم العنوان فقط، دون أي تفسير إضافي."""
  return generate_content(prompt)

def generate_book_outline(topic, style, num_chapters, author):
  prompt = f"""أنشئ مخططًا تفصيليًا لكتاب مكون من {num_chapters} فصول عن {topic} بأسلوب {style}، مكتوب بواسطة {author}.
  يجب أن يتضمن المخطط:
  1. مقدمة قصيرة لموضوع الكتاب
  2. لكل فصل:
     أ. عنوان شعري للفصل
     ب. ملخص قصير (2-3 جمل)
     ج. 3-5 عناوين فرعية أو أقسام رئيسية داخل الفصل
     د. نقاط الحبكة الرئيسية أو المفاهيم التي يجب تغطيتها
     هـ. تطوير الشخصيات أو العناصر الموضوعية التي يجب التركيز عليها
  3. قسم خاتمة أو ملحق
  استخدم لغة غنية وشعرية في جميع أنحاء المخطط. تأكد من وجود تقدم واضح ومنطقي للأفكار عبر الفصول."""
  return generate_content(prompt)

def generate_chapter(topic, style, chapter_number, chapter_outline, author):
  prompt = f"""اكتب الفصل {chapter_number} من كتاب عن {topic} بأسلوب {style}، مؤلف بواسطة {author}.
  استخدم المخطط التالي كدليل:

  {chapter_outline}

  يجب أن يكون الفصل حوالي 1000-1200 كلمة (حوالي 4 صفحات)، غنيًا باللغة الوصفية، والصور الحية، والعمق العاطفي.
  ركز على خلق جو قوي يعكس جوهر القصة.
  قم بتضمين الحوار، والمقاطع الوصفية، وأفكار الشخصيات كما هو مناسب للأسلوب والموضوع.

  هيكل الفصل مع أقسام واضحة:
  1. ابدأ بعنوان الفصل
  2. استخدم 3-5 عناوين فرعية خلال الفصل لتقسيم الأقسام الرئيسية
  3. تحت كل عنوان فرعي، اكتب 1-2 فقرات من المحتوى
  4. تأكد من وجود انتقالات سلسة بين الأقسام
  5. اختتم الفصل بخطاف أو مشوق قوي

  استخدم تراكيب جمل متنوعة وأطوال فقرات للحفاظ على تفاعل القارئ.
  تذكر أن تحافظ على الطول الإجمالي حوالي 4 صفحات (1000-1200 كلمة)."""
  return generate_content(prompt)

def generate_front_matter(book_title, author, year):
  prompt = f"""أنشئ مقدمة لكتاب بعنوان "{book_title}" بواسطة {author}، نُشر في {year}. تتضمن:
  1. إهداء قصير وجذاب (1-2 جمل)
  2. اقتباس شعري قصير (اقتباس يعكس جوهر الكتاب)
  3. مقدمة (حوالي 200 كلمة) تقدم موضوع الكتاب ودافع المؤلف
  4. قسم شكر وتقدير (حوالي 150 كلمة) يشكر الأفراد الرئيسيين أو مصادر الإلهام

  استخدم لغة تتناسب مع أسلوب وموضوع الكتاب.
  ابدأ كل قسم بعنوانه (مثل "الإهداء:"، "الاقتباس:"، إلخ) في سطر جديد."""
  front_matter = generate_content(prompt)

  # تهيئة القيم الافتراضية
  parsed_front_matter = {
      "dedication": "",
      "epigraph": "",
      "preface": "",
      "acknowledgments": ""
  }

  # تحليل المحتوى المُنشأ إلى أقسام
  current_section = None
  for line in front_matter.split('\n'):
      if line.lower().startswith("dedication:"):
          current_section = "dedication"
      elif line.lower().startswith("epigraph:"):
          current_section = "epigraph"
      elif line.lower().startswith("preface:"):
          current_section = "preface"
      elif line.lower().startswith("acknowledgments:"):
          current_section = "acknowledgments"
      elif current_section:
          parsed_front_matter[current_section] += line.strip() + "\n"

  # إزالة أي فواصل أسطر زائدة
  for key in parsed_front_matter:
      parsed_front_matter[key] = parsed_front_matter[key].strip()

  return parsed_front_matter

def generate_back_matter(book_title, author, topic):
  prompt = f"""أنشئ خاتمة لكتاب "{book_title}" بواسطة {author} عن {topic}. تتضمن:
  1. خاتمة (حوالي 300 كلمة) تعكس رحلة الكتاب وأهميته
  2. مسرد للمصطلحات أو المفاهيم الرئيسية المتعلقة بموضوع الكتاب (10-15 مصطلح)
  3. سيرة ذاتية قصيرة للمؤلف (حوالي 150 كلمة)
  4. اقتراحات للقراءة الإضافية (5-7 كتب أو مقالات متعلقة بالموضوع)

  تأكد من أن المحتوى جذاب ومرتبط بموضوع الكتاب.
  ابدأ كل قسم بعنوانه (مثل "الخاتمة:"، "المسرد:"، إلخ) في سطر جديد."""
  back_matter = generate_content(prompt)

  # تهيئة القيم الافتراضية
  parsed_back_matter = {
      "afterword": "",
      "glossary": "",
      "author_bio": "",
      "further_reading": ""
  }

  # تحليل المحتوى المُنشأ إلى أقسام
  current_section = None
  for line in back_matter.split('\n'):
      if line.lower().startswith("afterword:"):
          current_section = "afterword"
      elif line.lower().startswith("glossary:"):
          current_section = "glossary"
      elif line.lower().startswith("about the author:"):
          current_section = "author_bio"
      elif line.lower().startswith("further reading:"):
          current_section = "further_reading"
      elif current_section:
          parsed_back_matter[current_section] += line.strip() + "\n"

  # إزالة أي فواصل أسطر زائدة
  for key in parsed_back_matter:
      parsed_back_matter[key] = parsed_back_matter[key].strip()

  return parsed_back_matter

def sanitize_filename(filename):
  sanitized = re.sub(r'[<>:"/\|?*]', '', filename)
  return sanitized[:200]  # Limit to 200 characters

def write_book(author, year):
  topic = safe_input("أدخل موضوع الكتاب أو وصفه: ")
  style = safe_input("أدخل أسلوب الكتابة (مثل 'رواية', 'شعري'): ")
  num_chapters = int(safe_input("أدخل عدد الفصول للكتاب: "))

  book_title = generate_book_title(topic, style, author)
  print(f"عنوان الكتاب المُنشأ: {book_title}")

  print("جاري إنشاء مخطط الكتاب...")
  book_outline = generate_book_outline(topic, style, num_chapters, author)
  print("تم إنشاء مخطط الكتاب.")

  print("جاري إنشاء المقدمة...")
  front_matter = generate_front_matter(book_title, author, year)
  print("تم إنشاء المقدمة.")

  chapters = []
  for chapter in range(1, num_chapters + 1):
      print(f"جاري إنشاء الفصل {chapter}...")
      chapter_outline = extract_chapter_outline(book_outline, chapter)
      chapter_content = generate_chapter(topic, style, chapter, chapter_outline, author)
      chapters.append(chapter_content)
      time.sleep(random.uniform(2, 5))

  print("جاري إنشاء الخاتمة...")
  back_matter = generate_back_matter(book_title, author, topic)
  print("تم إنشاء الخاتمة.")

  return book_title, book_outline, front_matter, chapters, back_matter

def extract_chapter_outline(book_outline, chapter_number):
  chapter_start = book_outline.find(f"Chapter {chapter_number}")
  if chapter_start == -1:
      return "لم يتم العثور على مخطط الفصل."
  next_chapter = book_outline.find(f"Chapter {chapter_number + 1}", chapter_start)
  if next_chapter == -1:
      return book_outline[chapter_start:]
  return book_outline[chapter_start:next_chapter]

def organize_book_content(book_title, book_outline, front_matter, chapters, back_matter, author, year):
  organized_content = {
      "title": book_title,
      "author": author,
      "year": year,
      "front_matter": front_matter,
      "outline": book_outline,
      "chapters": [],
      "back_matter": back_matter
  }

  for i, chapter in enumerate(chapters, 1):
      chapter_content = {
          "number": i,
          "title": f"Chapter {i}",
          "sections": []
      }

      sections = re.split(r'\n(?=\d+\.)', chapter)
      for section in sections:
          if section.strip():
              lines = section.split('\n')
              section_title = lines[0].strip()
              section_content = '\n'.join(lines[1:]).strip()
              chapter_content["sections"].append({
                  "title": section_title,
                  "content": section_content
              })

      organized_content["chapters"].append(chapter_content)

  return organized_content

def add_page_number(paragraph):
  run = paragraph.add_run()
  fldChar = OxmlElement('w:fldChar')
  fldChar.set(qn('w:fldCharType'), 'begin')
  instrText = OxmlElement('w:instrText')
  instrText.set(qn('xml:space'), 'preserve')
  instrText.text = "PAGE"
  fldChar2 = OxmlElement('w:fldChar')
  fldChar2.set(qn('w:fldCharType'), 'end')
  run._element.append(fldChar)
  run._element.append(instrText)
  run._element.append(fldChar2)

def add_copyright_page(doc, author, year):
  doc.add_paragraph(f"حقوق الطبع والنشر © {year} بواسطة {author}")
  doc.add_paragraph("جميع الحقوق محفوظة.")
  doc.add_paragraph("لا يجوز إعادة إنتاج أي جزء من هذا الكتاب بأي شكل أو بأي وسيلة إلكترونية أو ميكانيكية، بما في ذلك أنظمة تخزين واسترجاع المعلومات، دون إذن كتابي من المؤلف، باستثناء استخدام الاقتباسات القصيرة في مراجعة الكتاب.")
  doc.add_page_break()

def add_headers_and_footers(doc, title, author):
  for section in doc.sections:
      header = section.header
      footer = section.footer

      # إضافة العنوان إلى الرأس
      header_para = header.paragraphs[0]
      header_para.text = title
      header_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT

      # إضافة المؤلف إلى التذييل
      footer_para = footer.paragraphs[0]
      footer_para.text = author
      footer_para.alignment = WD_ALIGN_PARAGRAPH.LEFT

      # إضافة رقم الصفحة إلى التذييل
      page_num_para = footer.add_paragraph()
      page_num_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
      add_page_number(page_num_para)

def save_book_as_docx(organized_content, filename=None):
  if filename is None:
      filename = sanitize_filename(organized_content["title"]) + ".docx"

  doc = Document()

  # إعداد حجم الصفحة A4
  section = doc.sections[0]
  section.page_height = Pt(842)  # ارتفاع A4 بالنقاط
  section.page_width = Pt(595)   # عرض A4 بالنقاط
  section.left_margin = Inches(1)
  section.right_margin = Inches(1)
  section.top_margin = Inches(1)
  section.bottom_margin = Inches(1)

  # إعداد الأنماط
  styles = doc.styles

  # نمط العنوان
  title_style = styles.add_style('CustomTitle', WD_STYLE_TYPE.PARAGRAPH)
  title_style.font.name = 'Garamond'
  title_style.font.size = Pt(36)
  title_style.font.bold = True
  title_style.font.color.rgb = RGBColor(0, 0, 128)  # أزرق بحري
  title_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
  title_style.paragraph_format.space_after = Pt(12)

  # نمط الفصل
  chapter_style = styles.add_style('CustomChapter', WD_STYLE_TYPE.PARAGRAPH)
  chapter_style.font.name = 'Georgia'
  chapter_style.font.size = Pt(24)
  chapter_style.font.bold = True
  chapter_style.font.color.rgb = RGBColor(128, 0, 0)  # خمري
  chapter_style.paragraph_format.space_before = Pt(24)
  chapter_style.paragraph_format.space_after = Pt(12)

  # نمط القسم
  section_style = styles.add_style('CustomSection', WD_STYLE_TYPE.PARAGRAPH)
  section_style.font.name = 'Calibri'
  section_style.font.size = Pt(18)
  section_style.font.bold = True
  section_style.font.color.rgb = RGBColor(0, 100, 0)  # أخضر داكن
  section_style.paragraph_format.space_before = Pt(18)
  section_style.paragraph_format.space_after = Pt(6)

  # نمط النص الأساسي
  body_style = styles.add_style('CustomBody', WD_STYLE_TYPE.PARAGRAPH)
  body_style.font.name = 'Bookman Old Style'
  body_style.font.size = Pt(11)
  body_style.paragraph_format.first_line_indent = Inches(0.25)
  body_style.paragraph_format.space_after = Pt(6)

  # نمط العنوان 1
  heading1_style = styles.add_style('CustomHeading1', WD_STYLE_TYPE.PARAGRAPH)
  heading1_style.font.name = 'Arial'
  heading1_style.font.size = Pt(20)
  heading1_style.font.bold = True
  heading1_style.font.color.rgb = RGBColor(70, 130, 180)  # أزرق فولاذي
  heading1_style.paragraph_format.space_before = Pt(24)
  heading1_style.paragraph_format.space_after = Pt(12)

  # صفحة العنوان
  doc.add_paragraph(organized_content["title"], style='CustomTitle')
  doc.add_paragraph(f"بواسطة {organized_content['author']}", style='CustomBody').alignment = WD_ALIGN_PARAGRAPH.CENTER
  doc.add_paragraph(str(organized_content["year"]), style='CustomBody').alignment = WD_ALIGN_PARAGRAPH.CENTER
  doc.add_page_break()

  # إضافة صفحة حقوق الطبع والنشر
  add_copyright_page(doc, organized_content['author'], organized_content['year'])

  # المقدمة
  doc.add_paragraph("الإهداء", style='CustomHeading1')
  doc.add_paragraph(organized_content["front_matter"]["dedication"], style='CustomBody')
  doc.add_paragraph("الاقتباس", style='CustomHeading1')
  doc.add_paragraph(organized_content["front_matter"]["epigraph"], style='CustomBody')
  doc.add_paragraph("المقدمة", style='CustomHeading1')
  doc.add_paragraph(organized_content["front_matter"]["preface"], style='CustomBody')
  doc.add_paragraph("الشكر والتقدير", style='CustomHeading1')
  doc.add_paragraph(organized_content["front_matter"]["acknowledgments"], style='CustomBody')
  doc.add_page_break()

  # جدول المحتويات
  doc.add_paragraph("جدول المحتويات", style='CustomHeading1')
  for chapter in organized_content["chapters"]:
      doc.add_paragraph(f"الفصل {chapter['number']}: {chapter['title']}", style='CustomBody')
  doc.add_page_break()

  # مخطط الكتاب
  doc.add_paragraph("مخطط الكتاب", style='CustomHeading1')
  doc.add_paragraph(organized_content["outline"], style='CustomBody')
  doc.add_page_break()

  # الفصول
  for chapter in organized_content["chapters"]:
      doc.add_paragraph(f"الفصل {chapter['number']}: {chapter['title']}", style='CustomChapter')
      for section in chapter["sections"]:
          doc.add_paragraph(section["title"], style='CustomSection')
          doc.add_paragraph(section["content"], style='CustomBody')
      doc.add_page_break()

  # الخاتمة
  doc.add_paragraph("الخاتمة", style='CustomHeading1')
  doc.add_paragraph(organized_content["back_matter"]["afterword"], style='CustomBody')
  doc.add_paragraph("المسرد", style='CustomHeading1')
  doc.add_paragraph(organized_content["back_matter"]["glossary"], style='CustomBody')
  doc.add_paragraph("عن المؤلف", style='CustomHeading1')
  doc.add_paragraph(organized_content["back_matter"]["author_bio"], style='CustomBody')
  doc.add_paragraph("قراءات إضافية", style='CustomHeading1')
  doc.add_paragraph(organized_content["back_matter"]["further_reading"], style='CustomBody')

  # إضافة الرؤوس والتذييلات
  add_headers_and_footers(doc, organized_content['title'], organized_content['author'])

  doc.save(filename)
  print(f"تم حفظ الكتاب باسم {filename}")

if __name__ == "__main__":
  print("مرحبًا بك في أداة كتابة الكتب المحسنة باستخدام Groq API ونموذج llama-3.1-70b-versatile!")
  try:
      author = safe_input("أدخل اسم المؤلف: ")
      year = safe_input("أدخل سنة الكتابة: ")
      book_title, book_outline, front_matter, chapters, back_matter = write_book(author, year)
      organized_content = organize_book_content(book_title, book_outline, front_matter, chapters, back_matter, author, year)
      save_book_as_docx(organized_content)
      print("اكتمل إنشاء الكتاب!")
  except Exception as e:
      print(f"حدث خطأ أثناء إنشاء الكتاب: {str(e)}")
      print("يرجى التحقق من مدخلاتك والمحاولة مرة أخرى.")

أدخل مفتاح Groq API الخاص بك: ··········
مرحبًا بك في أداة كتابة الكتب المحسنة باستخدام Groq API ونموذج llama-3.1-70b-versatile!
أدخل اسم المؤلف: الشيخ عبدالقادر بياي
أدخل سنة الكتابة: 2024
أدخل موضوع الكتاب أو وصفه: قصة احتلال موريتانيا 
أدخل أسلوب الكتابة (مثل 'رواية', 'شعري'): تاريخي
أدخل عدد الفصول للكتاب: 3
عنوان الكتاب المُنشأ: "أرض الدماء والغبار: ذكريات احتلال موريتانيا بين تاريخ وجرح مفتوح"
جاري إنشاء مخطط الكتاب...
تم إنشاء مخطط الكتاب.
جاري إنشاء المقدمة...
تم إنشاء المقدمة.
جاري إنشاء الفصل 1...
جاري إنشاء الفصل 2...
جاري إنشاء الفصل 3...
جاري إنشاء الخاتمة...
تم إنشاء الخاتمة.
تم حفظ الكتاب باسم أرض الدماء والغبار ذكريات احتلال موريتانيا بين تاريخ وجرح مفتوح.docx
اكتمل إنشاء الكتاب!
